In [1]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


In [20]:
def read_review_data(filename):
    with open(filename, 'r') as f:
        data=[line.split('\t') for line in f.read().splitlines()]
        data=data[1:]
    return data

In [21]:
start=time.time()

In [22]:
review_data=read_review_data('./ratings.txt')
print(len(review_data))

200000


In [23]:
print(review_data[0])

['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1']


In [24]:
komoran=Komoran()
docs= [komoran.nouns(sentence[1]) for sentence in review_data]

In [26]:
model=Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
model.save('nvmc.model')

In [27]:
print("corpus_count: ", model.corpus_count)
print("corpus_total_words : ",model.corpus_total_words)

corpus_count:  200000
corpus_total_words :  1076896


In [32]:
from gensim.models import Word2Vec 

#모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words: ",model.corpus_total_words)

#'사랑'이란 단어로 생성한 단어임베딩 벡터
#print('사랑: ',model.wv['사랑'])

#단어 유사도 계싼
print("일요일 = 월요일\t",model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t",model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t",model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t",model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t",model.wv.similarity(w1='히어로', w2='삼성'))

#가장 유사한 단어 추출
print(model.wv.most_similar("안성기",topn=5))
print(model.wv.most_similar("시리즈",topn=5))

corpus_total_words:  1076896
일요일 = 월요일	 0.6507668
안성기 = 배우	 0.5689335
대기업 = 삼성	 0.5282509
일요일 != 삼성	 0.16066258
히어로 != 삼성	 0.16068685
[('고준희', 0.7176288366317749), ('씨야', 0.7157871127128601), ('장미희', 0.7138710618019104), ('킬리언 머피', 0.7110390663146973), ('정재영', 0.7069952487945557)]
[('캐리비안의 해적', 0.6713436841964722), ('나니아 연대기', 0.6577498912811279), ('더 울버린', 0.6504627466201782), ('꽃보다 시리즈', 0.6454271674156189), ('X맨', 0.6427968144416809)]


n-gram 유사도 계산

In [33]:
from konlpy.tag import Komoran

#어절단위 n-gram
def word_ngram(bow,num_gram):
    text=tuple(bow)
    ngrams = [text[x:x +num_gram] for x in range(0,len(text))]
    return tuple(ngrams)

#유사도 계산
def similarity(doc1, doc2):
    cnt = 0
    for token in doc1:
        if token in doc2:
            cnt = cnt + 1
    return cnt/len(doc1)

In [34]:
#문장정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

komoran = Komoran()
bow1= komoran.nouns(sentence1)
bow2= komoran.nouns(sentence2)
bow3= komoran.nouns(sentence3)

#단어 n-gram 토큰 추출
doc1 = word_ngram(bow1,2)  #2-gram방식으로 추출
doc2 = word_ngram(bow2,2)
doc3 = word_ngram(bow3,2)

#추출된 n-gram 토큰 출력
print(doc1)
print(doc2)

#유사도 계산
r1=similarity(doc1, doc2)
r2=similarity(doc3, doc1)

#계산된 유사도 출력
print(r1)
print(r2)


(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
0.6666666666666666
0.0


3개 문장간의 코사인 유사도 계산

In [35]:
from konlpy.tag import Komoran 
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [38]:
#코사인 유사도 계산
def cos_sim(vec1,vec2):
    return dot(vec1,vec2) /(norm(vec1)*norm(vec2))

#TDM만들기
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}

    for word in word_dics:
        freq_mat[word]=0

    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word]+= 1

    return freq_mat

#단어 벡터word vector만들기
def make_vector(tdm):
    vec=[]
    for key in tdm:
        vec.append(tdm[key])
    return vec


In [39]:
#문장정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

komoran = Komoran()
bow1= komoran.nouns(sentence1)
bow2= komoran.nouns(sentence2)
bow3= komoran.nouns(sentence3)

#단어묶음 리스트를 하나로 합침
bow= bow1+bow2+bow3

#단어묶음에서 중복을 제거해 단어 사전 구축
word_dics=[]
for token in bow:
    if token not in word_dics:
        word_dics.append(token)

In [40]:
#문장별 단어 문서 행렬 계산
freq_list1=make_term_doc_mat(bow1,word_dics)
freq_list2=make_term_doc_mat(bow2,word_dics)
freq_list3=make_term_doc_mat(bow3,word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}


In [47]:
#문장 벡터 생성
doc1=np.array(make_vector(freq_list1))
doc2=np.array(make_vector(freq_list2))
doc3=np.array(make_vector(freq_list3))
print(doc1)
print(doc2)
print(doc3)

[1 1 1 1 1 1 0 0 0 0]
[1 1 1 1 0 1 1 0 0 0]
[0 1 0 0 0 0 0 1 1 1]


In [44]:
#코사인 유사도 계산
r1= cos_sim(doc1,doc2)
r2= cos_sim(doc3,doc1)
print(r1)
print(r2)

0.8333333333333335
0.20412414523193154
